<a href="https://colab.research.google.com/github/AliKarahaliloz/Fake-News-Detection-With-Bert/blob/main/fake_news_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install libraries
!pip install transformers datasets accelerate peft bitsandbytes

import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
import torch

!apt-get update && apt-get install -y libstdc++6
import os
!pip install --upgrade bitsandbytes

!pip install --upgrade transformers
!pip install --upgrade peft

# install dataset
!kaggle datasets download -d emineyetm/fake-news-detection-datasets
!unzip fake-news-detection-datasets.zip

#save dataset
true_df = pd.read_csv("News _dataset/True.csv")
fake_df = pd.read_csv("News _dataset/Fake.csv")

#labelling
true_df["label"] = 1
fake_df["label"] = 0

#unite the datasets and shuffle
df = pd.concat([true_df, fake_df], axis=0)
df = df.sample(frac=1).reset_index(drop=True)

#turn into huggingface format
dataset = Dataset.from_pandas(df[["text", "label"]])
dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset["train"]
test_dataset = dataset["test"]


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 128 kB in 2s (61.7 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

In [ ]:
#print the first 55 of the dataset
df.head(55)

,title,text,subject,date,label
0,Trump adds antitrust expert to Justice transit...,WASHINGTON (Reuters) - A Republican antitrust ...,politicsNews,"November 28, 2016",1
1,"Trump talks Paris agreement, Iran with France'...",NEW YORK (Reuters) - U.S. President Donald Tru...,worldnews,"September 18, 2017",1
2,Ann Coulter Goes Full Bigot – Calls Khizr Kha...,Conservative commentator and devout Trump love...,News,"September 6, 2016",0
3,THE COMMUNITY ORGANIZER Who Won’t Go Away: Oba...,Ed Klein: For the past 100 years every presid...,left-news,"Dec 18, 2016",0
4,JOHN KERRY GRILLED BY GREAT PATRIOT TOM COTTON...,WOW! TOM COTTON KNOW HIS INFO! Please listen a...,Government News,"Jul 30, 2015",0
5,Saudis pledge $100 million to African anti-jih...,DAKAR (Reuters) - Saudi Arabia has pledged $10...,worldnews,"December 4, 2017",1
6,U.S. retailers bet on Congress over Bolivia to...,"CHICAGO (Reuters) - Target Corp (TGT.N), Wal-M...",politicsNews,"April 21, 2017",1
7,Senate votes to confirm slate of three CFTC co...,WASHINGTON (Reuters) - The U.S. Senate voted o...,politicsNews,"August 3, 2017",1
8,Desperate Lindsey Graham Calls Clinton Founda...,"Senator Lindsey Graham has had an on-again, of...",News,"August 24, 2016",0
9,"Trump Can Only Think Of One Qualified Woman, ...","If you recall the 2012 election, which seemed ...",News,"August 5, 2016",0


In [ ]:
# load the model and thetokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenizer function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

# Tokenizing the dataset
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/8980 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType


# Lora config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

#wrap the model with lora
model = get_peft_model(model, lora_config)

# Optimizer
optimizer = AdamW(
    [
        {'params': [p for n, p in model.named_parameters() if 'lora' in n], 'lr': 2e-5, 'weight_decay': 0.01},  # LoRA parameters
        {'params': [p for n, p in model.named_parameters() if 'lora' not in n], 'lr': 1e-5, 'weight_decay': 0.001}  # Other parameters
    ]
)


# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=1e-6,
    weight_decay=0.01,
    fp16=True,

    max_steps=6000,
    report_to="wandb",


    optim="adamw_torch",

)

# Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    optimizers=(optimizer, None)
)

# Start the training
trainer.train()

# Save the model
trainer.save_model("./fake_news_detection_model")



/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alikarahaliloz (alikarahaliloz78) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
0,0.142500,0.083901


In [ ]:
#download the model
!zip -r /content/fake_news_detection_model.zip /content/fake_news_detection_model
from google.colab import files
files.download('/content/fake_news_detection_model.zip')



  adding: content/fake_news_detection_model/ (stored 0%)
  adding: content/fake_news_detection_model/training_args.bin (deflated 51%)
  adding: content/fake_news_detection_model/adapter_config.json (deflated 54%)
  adding: content/fake_news_detection_model/README.md (deflated 66%)
  adding: content/fake_news_detection_model/adapter_model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#zipping the others
!zip -r /content/tum_klasorler.zip /content/fake_news_detection_model /content/results /content/wandb

  adding: content/fake_news_detection_model/ (stored 0%)
  adding: content/fake_news_detection_model/training_args.bin (deflated 51%)
  adding: content/fake_news_detection_model/adapter_config.json (deflated 54%)
  adding: content/fake_news_detection_model/README.md (deflated 66%)
  adding: content/fake_news_detection_model/adapter_model.safetensors (deflated 7%)
  adding: content/results/ (stored 0%)
  adding: content/results/checkpoint-6000/ (stored 0%)
  adding: content/results/checkpoint-6000/optimizer.pt (deflated 7%)
  adding: content/results/checkpoint-6000/training_args.bin (deflated 51%)
  adding: content/results/checkpoint-6000/adapter_config.json (deflated 54%)
  adding: content/results/checkpoint-6000/trainer_state.json (deflated 58%)
  adding: content/results/checkpoint-6000/rng_state.pth (deflated 25%)
  adding: content/results/checkpoint-6000/scheduler.pt (deflated 55%)
  adding: content/results/checkpoint-6000/README.md (deflated 66%)
  adding: content/results/checkpoin

In [ ]:
#downloading the others
from google.colab import files
files.download('/content/tum_klasorler.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#start to test our model

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# Install the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


# Install the adapter
peft_config = PeftConfig.from_pretrained("./fake_news_detection_model")

# Install the adapter weights
model = PeftModel.from_pretrained(model1, "./fake_news_detection_model", config=peft_config)

# Create pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# News text
news_text = "Trump administration issues new rules on U.S. visa waivers."

# Predict
result = classifier(news_text)

# Print
print(result)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DiffLlamaForSequenceClassific

[{'label': 'LABEL_1', 'score': 0.9864104390144348}]


In [ ]:
#testing with true news

import torch
# Test text
text = "Trump administration issues new rules on U.S. visa waivers." # True news

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
outputs = model(**inputs)

# Get the output
logits = outputs.logits
prediction = torch.argmax(logits, dim=-1)  # Predict (0: Fake, 1: True)
print("Prediction:", prediction.item())  # Print 0 or 1



Prediction: 1


In [ ]:
#testing with fake news

import torch
# Test text
text = "WATCH: George W. Bush Calls Out Trump For Supporting White Supremacy." # Fake news

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
outputs = model(**inputs)

# Get the output
logits = outputs.logits
prediction = torch.argmax(logits, dim=-1)  # Predict (0: Fake, 1: True)
print("Prediction:", prediction.item())  # Print 0 or 1





Prediction: 0
